<font size="5"> **WEB SCRAPER - LABMOL** </font> 


<font size="2"> by: Igor Sanches </font> 

**READ BEFORE CONTINUING!!**

Please generate your api key at https://bit.ly/3dZC7VT (you will need it)

**choose your keywords in the following format "word1+word2+...+word5"**

choose a folder path to save the results file, see how: https://bit.ly/3sgLJUk


In [1]:
%%capture
#@title <-----  Install dependencies { vertical-output: true }
!pip install scholarly
!pip install requests
!pip install beautifulsoup4
!pip install pandas


In [2]:
%%capture
from google.colab import drive
drive.mount('/content/drive')

###### add your folder path here ######
Save_Location = '/content/drive/MyDrive/' #type here the path where the results will be saved.
                                                      #do not add '/' at the end of the path.

In [3]:
#@title <- run
API_KEY = input("type your API key: ").strip()
keywords = input("keywords: ").strip().lower()
print("this will automatically save the results to a csv file at the chosen location")
output = input("how do you wish to name the file? ").strip()
tables = []

import requests
from bs4 import BeautifulSoup
import concurrent.futures
import csv
import urllib.parse
import pandas as pd
from scholarly import scholarly, ProxyGenerator
import re

type your API key: jasdijasdihasluihasdijhasdasd
keywords: tomate+vermelho+genoma
this will automatically save the results to a csv file at the chosen location
how do you wish to name the file? primeira_busca


# **GOOGLE SCHOLAR**



In [ ]:
#@title <- start { vertical-output: true }

pages = 99

pg = ProxyGenerator()
success = pg.ScraperAPI(API_KEY)

#number of requests to be tried in case of failure 
NUM_RETRIES = 3

#number of threads to be executed concurrently
NUM_THREADS = 5

#standard google scholar general search url
urlgo = f"https://scholar.google.com/scholar?start=0&q={keywords}&hl=en&as_sdt=0,5"
list_of_urls = []

#generates gs urls based on the amount of pages the user requested
if pages == 1:
  list_of_urls = urlgo
else:
  for page in range(pages):
    list_of_urls.append(f'https://scholar.google.com/scholar?start={(page*10)}&q={keywords}&hl=en&as_sdt=0,5')

#scraped data will be saved here
df0 = []

def scrape_url(url):
    ##### SCRAPE FUNCTION  #####
      
    params = {'api_key': API_KEY, 'url': url}
    
    # send request to scraperapi, and automatically retry failed requests
    for _ in range(NUM_RETRIES):
        try:
            response = requests.get('http://api.scraperapi.com/', params = urllib.parse.urlencode(params))
            if response.status_code in [200, 404]:
                ## escape for loop if the API returns a successful response
                break
        except requests.exceptions.ConnectionError:
            response = ''
      
      
    ## parse data ONLY if 200 status code (successful response)
    if response.status_code == 200:
          
        ## parse syntax
        html_response = response.text
        soup = BeautifulSoup(html_response, "html.parser")
          
        #requesting title, link and year
        for item in soup.select('[data-lid]'):
          title = item.select("h3")[0].get_text().replace('[HTML]', '')
          link = item.select("a")[0]["href"]
          year = item.select(".gs_a")[0]
          yearstripped = year.get_text()

          #retrives the year using regex
          finalyear = re.search(r"(\d{4})", yearstripped).group()
          df0.append({
                'title': title,
                'link': link,
                'year': finalyear
            })

#running multiple threads to speed up the scraping process  
with concurrent.futures.ThreadPoolExecutor(max_workers=NUM_THREADS) as executor:
    executor.map(scrape_url, list_of_urls)

df0 = pd.DataFrame(df0)
print('DONE')
tables.append(df0)

# **PUBMED**

In [ ]:
#@title <- start { vertical-output: true }

pages = 99

pg = ProxyGenerator()
success = pg.ScraperAPI(API_KEY)

#number of requests to be tried in case of failure 
NUM_RETRIES = 3

#number of threads to be executed concurrently
NUM_THREADS = 5

keywords = keywords.replace('+', '%20')

#standard pubmed general search url
urlgo = f"https://pubmed.ncbi.nlm.nih.gov/?term={keywords}"


#generates gs urls based on the amount of pages the user requested
list_of_urls = []
if pages == 1:
  list_of_urls = urlgo
else:
  for page in range(pages):
    list_of_urls.append(f'https://pubmed.ncbi.nlm.nih.gov/?term={keywords}&page={page+1}')

#scraped data will be saved here
df1 = []

def scrape_url(url):
    ##### SCRAPE FUNCTION  #####
      
    params = {'api_key': API_KEY, 'url': url}
    
    # send request to scraperapi, and automatically retry failed requests
    for _ in range(NUM_RETRIES):
        try:
            response = requests.get('http://api.scraperapi.com/', params = urllib.parse.urlencode(params))
            if response.status_code in [200, 404]:
                ## escape for loop if the API returns a successful response
                break
        except requests.exceptions.ConnectionError:
            response = ''
      
      
    ## parse data ONLY if 200 status code (successful response)
    if response.status_code == 200:
          
        ## parse syntax
        html_response = response.text
        soup = BeautifulSoup(html_response, "html.parser")
          
        #requesting title, link and year
        for item in soup.select('[data-rel-pos]'):
          title = item.select("a")[0].get_text().replace("\n", "").strip()

          linknumber = item.select("a")[0]["href"]
          link = f'https://pubmed.ncbi.nlm.nih.gov{linknumber}'

          year = item.select(".docsum-journal-citation.full-journal-citation")[0]
          yearstripped = year.get_text()

          #retrives the year using regex
          finalyear = re.search(r"(\d{4})", yearstripped).group()
          df1.append({
                'title': title,
                'link': link,
                'year': finalyear
            })
          

#running multiple threads to speed up the scraping process  
with concurrent.futures.ThreadPoolExecutor(max_workers=NUM_THREADS) as executor:
    executor.map(scrape_url, list_of_urls)

df1 = pd.DataFrame(df1)
print('DONE')
tables.append(df1)

# **SEMANTICS SCHOLAR, SCIELO, WEB OF SCIENCE & SCIFINDER**



In [ ]:
#@title <- start { vertical-output: true }

pg = ProxyGenerator()
success = pg.ScraperAPI(API_KEY)

#number of requests to be tried in case of failure 
NUM_RETRIES = 3

#number of threads to be executed concurrently
NUM_THREADS = 5

#standard semantics scholar general search url
baseurl = f'https://api.semanticscholar.org/graph/v1/paper/search?query={keywords}&offset=0&limit=100&fields=title,year,url'

#pre request to check the amount of papers to be scraped
r = requests.get(baseurl)
total = r.json()['total']
total = int(total)//100
list_of_urls = []

#generates the list of urls
for i in range(total+1):
  list_of_urls.append(f'https://api.semanticscholar.org/graph/v1/paper/search?query={keywords}&offset={i*100}&limit=100&fields=title,year,url')



#scraped data will be saved here
df2 = []

def scrape_url(url):
    ##### SCRAPE FUNCTION  #####
      
    params = {'api_key': API_KEY, 'url': url}
    
    # send request to scraperapi, and automatically retry failed requests
    for _ in range(NUM_RETRIES):
        try:
            response = requests.get('http://api.scraperapi.com/', params = urllib.parse.urlencode(params))
            if response.status_code in [200, 404]:
                ## escape for loop if the API returns a successful response
                break
        except requests.exceptions.ConnectionError:
            response = ''
      
      
    ## parse data ONLY if 200 status code (successful response)
    if response.status_code == 200:

      data = response.json()
      for i in data['data']:
        title = i['title']
        year = i['year']
        link = i['url']

        df2.append({
          'title':title,
          'link':link,
          'year':year
        })

#running multiple threads to speed up the scraping process  
with concurrent.futures.ThreadPoolExecutor(max_workers=NUM_THREADS) as executor:
    executor.map(scrape_url, list_of_urls)

df2 = pd.DataFrame(df2)
print("DONE")
tables.append(df2)

=-=-=-=-=-=-=-=-=--=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=--=-=-=-=-=-=-=-=-=-=-=-=-=-

# **SAVE AND EXPORT TO DRIVE**

In [ ]:
#@title <-  run { vertical-output: true }
result = pd.concat(tables)
result.to_excel(f"{Save_Location}/{output}.xlsx", sheet_name= "webscraping")
print("DONE")
pd.DataFrame(result)